In [28]:
import pandas as pd
import numpy as np
import pathlib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from gosdt import ThresholdGuessBinarizer, GOSDTClassifier, NumericBinarizer

# Example 1

GOSDT runs on binary datasets. In this example, we run GOSDT on a binarized Monk 1 dataset. We download Monk 1 from the UCI data repository and use a one-hot encoding from scikit-learn to transform categorical features to binary features.

In [29]:
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/monks-problems/monks-1.train", sep=" ", header=None
)
df = df.iloc[:,1:-1]

# Assign column names 
df.columns = ["label", "head_shape", "body_shape", "is_smiling", "holding", "jacket_color", "has_tie"]
X = df.iloc[:,1:]
y = df.iloc[:,0]

# Encode the categorical features to binary features
enc = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
X_new = enc.fit_transform(X)
X_new

,head_shape_1,head_shape_2,head_shape_3,body_shape_1,body_shape_2,body_shape_3,is_smiling_1,is_smiling_2,holding_1,holding_2,holding_3,jacket_color_1,jacket_color_2,jacket_color_3,jacket_color_4,has_tie_1,has_tie_2
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
120,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
121,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
122,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


Set GOSDT Configuration and fit the model. GOSDT finds a 7-leaf tree with 100% accuracy within seconds.

In [30]:
clf = GOSDTClassifier(regularization=0.02, depth_budget=6, time_limit=60, similar_support=False)
clf.fit(X_new, y)

print("Evaluating the model, extracting the tree and scores")
results = clf.get_result()

print(f"Model training time: {results["time"]}")
print(f"Training accuracy: {clf.score(X_new, y)}")

Evaluating the model, extracting the tree and scores
Model training time: 0.507
Training accuracy: 1.0


# Example 2

In our first example, we showed that GOSDT can solve a small dataset (Monk 1) in seconds. In this example, we run GOSDT on COMPAS, a recidivism dataset. The COMPAS dataset is much larger, containing 6907 samples and 7 continuous features.

In [31]:
df = pd.read_csv("../datasets/compas.csv")
X, y = df.iloc[:,:-1], df.iloc[:,-1]
X_bin = NumericBinarizer().set_output(transform="pandas").fit_transform(X, y)
df

,sex=female,age,juv_fel_count,juv_misd_count,juvenile_crimes,priors_count,current_charge_degree=felony,two_year_recid
0,0,69,0,0,0,0,1,0
1,0,34,0,0,0,0,1,1
2,0,24,0,0,1,4,1,1
3,0,44,0,0,0,0,0,0
4,0,41,0,0,0,14,1,1
...,...,...,...,...,...,...,...,...
6902,0,23,0,0,0,0,1,0
6903,0,23,0,0,0,0,1,0
6904,0,57,0,0,0,0,1,0
6905,1,33,0,0,0,3,0,0


To start, we fit a simple model with a small depth constraint and large regularization. Even for a complicated datset, GOSDT can efficiently find an optimal tree.

In [32]:
clf = GOSDTClassifier(regularization=0.01, depth_budget=4, time_limit=60, similar_support=False)
clf.fit(X_bin, y)

print("Evaluating the model, extracting the tree and scores")
results = clf.get_result()

print(f"Model training time: {results["time"]}")
print(f"Training accuracy: {clf.score(X_bin, y)}")
print(f"Number of iterations: {results["n_iterations"]}")

Evaluating the model, extracting the tree and scores
Model training time: 9.893
Training accuracy: 0.6632401911104676
Number of iterations: 37119


In the above, we had to constrain our search to very simple trees. We're likely to get a more useful and accurate moedl when we allow our model to find more complicated trees (we do this with less constraint on depth and with smaller regularization). However, for real-valued datasets like COMPAS, GOSDT might not finish running within a couple minutes. An example is shown below where the depth budget is set to 6 and the regularization is set to 0.001 for the COMPAS dataset. After 60 seconds, GOSDT hits the time limit and returns the current best model.

In [33]:
clf = GOSDTClassifier(regularization=0.001, depth_budget=6, time_limit=60, similar_support=False)
clf.fit(X_bin, y)

print("Evaluating the model, extracting the tree and scores")
results = clf.get_result()

print(f"Model training time: {results["time"]}")
print(f"Training accuracy: {clf.score(X_bin, y)}")
print(f"Number of iterations: {results["n_iterations"]}")

Possible timeout: 62.8 Queue Size: 2849413
[WARNING] Optimization did not finish within the time limit, the result may not be optimal.
Evaluating the model, extracting the tree and scores
Model training time: 62.8
Training accuracy: 0.6667149268857681
Number of iterations: 139999


With the time-out, GOSDT didn't find a very accurate tree because it couldn't finish optimizing. If it did finish optimizing, we might need to wait for a long time or use a large amount of memory. An alternative, presented in [our AAAI '22 paper](https://arxiv.org/abs/2112.00798), is to use threshold and lowerbound guesses to help GOSDT find a near optimal tree for COMPAS within 1 minute. The guessing process works as follows: 

### 1. Threshold guessing: we first use a black box model (gradient boosted trees) to guess useful thresholds

One of the biggest slowdowns for decision trees is deciding how to binarize the continuous features. We had a separate colmn in our dataset for age <= 20.5, age <= 21.5, age <= 25.5, ... and so on. Using threshold guessing lets us prune the thresholds we use, by looking at which thresholds are used by gradient boosted trees.

In [34]:
enc = ThresholdGuessBinarizer(n_estimators=40, max_depth=1, random_state=2021).set_output(transform="pandas")
X_guessed = enc.fit_transform(X, y)
X_guessed

,age <= 20.5,age <= 22.5,age <= 27.5,age <= 29.5,age <= 33.5,age <= 34.5,age <= 36.5,juvenile_crimes <= 0.5,priors_count <= 0.5,priors_count <= 1.5,priors_count <= 2.5,priors_count <= 3.5,priors_count <= 5.5,priors_count <= 7.5,priors_count <= 8.5,age <= 23.5
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6902,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6903,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
6905,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0


### 2. Lower bound guessing: we use gradient boosted decision trees to get labels for the training dataset

We determine what labels the gradient boosted tree model would predict for each of the training points. The idea is that many good decision trees are likely to make similar predictions to the gradient boosted tree on the training data.

In [35]:
clf = GradientBoostingClassifier(n_estimators=40, max_depth=1, random_state=42)
clf.fit(X_guessed, y)
y_ref = clf.predict(X_guessed)

### 3. Train GOSDT model

We can use the information from our guesses to speed up training GOSDT, while retaining some nice properties on the tree it returns (see our AAAI '22 paper for more details).

In [36]:
clf = GOSDTClassifier(regularization=0.001, depth_budget=6, time_limit=60, similar_support=False, verbose=True)
clf.fit(X_guessed, y, y_ref=y_ref)

Using Configuration: {
    "cancellation": true,
    "depth_budget": 6,
    "diagnostics": false,
    "feature_transform": true,
    "look_ahead": true,
    "model_limit": 1,
    "non_binary": false,
    "profile": "",
    "reference_LB": false,
    "regularization": 0.0010000000474974513,
    "rule_list": false,
    "similar_support": false,
    "time_limit": 60,
    "trace": "",
    "tree": "",
    "upperbound": 0.0,
    "verbose": true,
    "worker_limit": 1
}
Initializing Optimization Framework.
Starting Optimization.

GOSDTClassifier(depth_budget=6, regularization=0.001, similar_support=False,
                time_limit=60, verbose=True)


Time: 0, Objective: [0.312975, 0.356857], Boundary: 0.043882, Graph Size: 1, Queue Size: 32
Time: 0.509, Objective: [0.312975, 0.356857], Boundary: 0.043882, Graph Size: 3674, Queue Size: 6300
Time: 0.738, Objective: [0.313975, 0.32274], Boundary: 0.00876427, Graph Size: 4812, Queue Size: 495
Time: 0.752, Objective: [0.316265, 0.32274], Boundary: 0.00647473, Graph Size: 4827, Queue Size: 9
Time: 0.752, Objective: [0.319305, 0.32274], Boundary: 0.0034343, Graph Size: 4827, Queue Size: 10
Time: 0.752, Objective: [0.321016, 0.32274], Boundary: 0.00172389, Graph Size: 4827, Queue Size: 8
Time: 0.752, Objective: [0.322147, 0.32274], Boundary: 0.00059256, Graph Size: 4827, Queue Size: 7
Time: 0.752, Objective: [0.322292, 0.32274], Boundary: 0.00044781, Graph Size: 4827, Queue Size: 7
Time: 0.752, Objective: [0.32274, 0.32274], Boundary: 0, Graph Size: 4827, Queue Size: 6
Optimization Complete.
Training Duration: 0.752
Number of Optimizer Iterations: 18493
Size of Problem Graph: 4827
Objecti

### 4. Get results

In [37]:
print(f"Training accuracy: {clf.score(X_guessed, y)}")
for tree in clf.trees_:
    print(tree)

Training accuracy: 0.6862603156218329
<class 'gosdt._tree.Tree'>: { feature: 0 [ left child: { prediction: 1, loss: 0.007383813615888357 }, right child: { feature: 2 [ left child: { feature: 9 [ left child: { feature: 1 [ left child: { feature: 7 [ left child: { prediction: 0, loss: 0.025191834196448326 }, right child: { prediction: 1, loss: 0.0047777616418898106 }] }, right child: { prediction: 0, loss: 0.03793253377079964 }] }, right child: { prediction: 1, loss: 0.041262488812208176 }] }, right child: { feature: 6 [ left child: { feature: 10 [ left child: { prediction: 0, loss: 0.04647459089756012 }, right child: { prediction: 1, loss: 0.04705371335148811 }] }, right child: { feature: 13 [ left child: { prediction: 0, loss: 0.07948458194732666 }, right child: { prediction: 1, loss: 0.024178368970751762 }] }] }] }] }, Index(['age <= 20.5', 'age <= 22.5', 'age <= 27.5', 'age <= 29.5',
       'age <= 33.5', 'age <= 34.5', 'age <= 36.5', 'juvenile_crimes <= 0.5',
       'priors_count <=

After using the threshold and lower bound guesses, GOSDT can find a near optimal tree within seconds. This tree is much more accurate than the tree we found earlier, when using much more restrictive parameters.

# Example 3
In this example, we show a comprehensive modeling and evaluation process using GOSDT with guesses on the COMPAS dataset. This example includes how to split data into training and test set and how to get training and test set after threshold guessing. 

For this example, we start by using guessing immediately, which is what we recommend you do if you have an extraordinarily large number of features. We _could_ have followed the same process as in example 2 (first run without guessing, then see what happens and use guessing if we time out). However, there are so many features here that we will definitely need guessing to find accurate trees in reasonable time and memory. If we tried running without guessing, we would be likely to kill the kernel because of how much memory we would be using!

In [38]:
# read the dataset
df = pd.read_csv("../datasets/compas.csv")
X, y = df.iloc[:,:-1], df.iloc[:,-1]
h = df.columns[:-1]
df

,sex=female,age,juv_fel_count,juv_misd_count,juvenile_crimes,priors_count,current_charge_degree=felony,two_year_recid
0,0,69,0,0,0,0,1,0
1,0,34,0,0,0,0,1,1
2,0,24,0,0,1,4,1,1
3,0,44,0,0,0,0,0,0
4,0,41,0,0,0,14,1,1
...,...,...,...,...,...,...,...,...
6902,0,23,0,0,0,0,1,0
6903,0,23,0,0,0,0,1,0
6904,0,57,0,0,0,0,1,0
6905,1,33,0,0,0,3,0,0


In [39]:
# If train-test split is desired
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)
print(X_train.shape)
print(X_test.shape)

(5525, 7)
(1382, 7)


#### 1. Threshold guess: we first use the gradient boosted tree to guess useful thresholds. 

In [40]:
enc = ThresholdGuessBinarizer(n_estimators=40, max_depth=1, random_state=2021).set_output(transform="pandas")
X_train_guessed = enc.fit_transform(X_train, y_train)
X_test_guessed = enc.transform(X_test)
print(X_train_guessed.shape)
print(X_test_guessed.shape)
print("")

(5525, 17)
(1382, 17)


### 2. Lower bound guessing: we use gradient boosted decision trees to get labels for the training dataset

In [45]:
clf = GradientBoostingClassifier(n_estimators=40, max_depth=1, random_state=42)
clf.fit(X_train_guessed, y_train)
y_ref = clf.predict(X_train_guessed)

### 3. Train GOSDT model

In [46]:
clf = GOSDTClassifier(regularization=0.001, depth_budget=5, time_limit=60, similar_support=False, verbose=True)
clf.fit(X_train_guessed, y_train, y_ref=y_ref)

Using Configuration: {
    "cancellation": true,
    "depth_budget": 5,
    "diagnostics": false,
    "feature_transform": true,
    "look_ahead": true,
    "model_limit": 1,
    "non_binary": false,
    "profile": "",
    "reference_LB": false,
    "regularization": 0.0010000000474974513,
    "rule_list": false,
    "similar_support": false,
    "time_limit": 60,
    "trace": "",
    "tree": "",
    "upperbound": 0.0,
    "verbose": true,
    "worker_limit": 1
}
Initializing Optimization Framework.
Starting Optimization.


GOSDTClassifier(depth_budget=5, regularization=0.001, similar_support=False,
                time_limit=60, verbose=True)

Time: 0, Objective: [0.315579, 0.357475], Boundary: 0.0418959, Graph Size: 1, Queue Size: 34
Time: 0.247, Objective: [0.316579, 0.325914], Boundary: 0.00933486, Graph Size: 2089, Queue Size: 72
Time: 0.248, Objective: [0.316579, 0.325914], Boundary: 0.00933483, Graph Size: 2089, Queue Size: 51
Time: 0.248, Objective: [0.316941, 0.325914], Boundary: 0.00897282, Graph Size: 2089, Queue Size: 34
Time: 0.248, Objective: [0.325104, 0.325914], Boundary: 0.000809938, Graph Size: 2089, Queue Size: 3
Time: 0.248, Objective: [0.325742, 0.325914], Boundary: 0.00017193, Graph Size: 2089, Queue Size: 2
Time: 0.248, Objective: [0.325914, 0.325914], Boundary: 0, Graph Size: 2089, Queue Size: 0
Optimization Complete.
Training Duration: 0.248
Number of Optimizer Iterations: 6309
Size of Problem Graph: 2089
Objective Boundary: [0.325914, 0.325914]
Models Generated: 1
Loss: 0.318914
Complexity: 0.007


#### 4. Get results

In [50]:
print(f"Model trainint time: {clf.get_result()["time"]}")
print(f"Training accuracy: {clf.score(X_train_guessed, y_train)}")
print(f"Test accuracy: {clf.score(X_test_guessed, y_test)}")
for tree in clf.trees_:
    print(tree)

Model trainint time: 0.248
Training accuracy: 0.6810859728506787
Test accuracy: 0.6903039073806078
<class 'gosdt._tree.Tree'>: { feature: 1 [ left child: { prediction: 1, loss: 0.025520361959934235 }, right child: { feature: 3 [ left child: { feature: 10 [ left child: { prediction: 0, loss: 0.05266968160867691 }, right child: { prediction: 1, loss: 0.03981900215148926 }] }, right child: { feature: 7 [ left child: { feature: 16 [ left child: { prediction: 0, loss: 0.06787329912185669 }, right child: { prediction: 1, loss: 0.04488687589764595 }] }, right child: { feature: 14 [ left child: { prediction: 0, loss: 0.06696832180023193 }, right child: { prediction: 1, loss: 0.021176470443606377 }] }] }] }] }, Index(['age <= 20.5', 'age <= 21.5', 'age <= 22.5', 'age <= 27.5',
       'age <= 32.5', 'age <= 33.5', 'age <= 34.5', 'age <= 38.5',
       'juvenile_crimes <= 0.5', 'priors_count <= 0.5', 'priors_count <= 1.5',
       'priors_count <= 2.5', 'priors_count <= 4.5', 'priors_count <= 5.5',

As you can see, we find a reasonably accurate model quite quickly on even larger datasets!

Thank you for reading our tutorial. Please do try out our methods with different parameters and datasets. Happy tree training!